In [67]:
%%capture
pip install cobra

# General Add reaction methods

In [68]:
#create a Reaction 
#reaction = Reaction('R_3OAS140')
#reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
#reaction.subsystem = 'Cell Envelope Biosynthesis'
#reaction.lower_bound = 0.  # This is the default
#reaction.upper_bound = 1000.  # This is the default

In [69]:
#Create metabolites
#ACP_c = Metabolite(
    #'ACP_c',
    #formula='C11H21N2O7PRS',
    #name='acyl-carrier-protein',
    #compartment='c')

In [70]:
#ADD metabolites to reactions
#reaction.add_metabolites({
    #malACP_c: -1.0,
    #h_c: -1.0,
    #ddcaACP_c: -1.0,
    #co2_c: 1.0,
    #ACP_c: 1.0,
    #omrsACP_c: 1.0
#})

In [71]:
#Add Gene reaction rules
#reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
#reaction.genes

In [72]:
#Add the reaction to the model

#model.add_reactions([reaction])#

# **Preparation and data import** 


In [73]:
import cobra

In [74]:
Model_path = '/content/drive/MyDrive/Halomonas_Model_devt/Halomonas_RASTV1.xml'
#Model_path = "/content/drive/MyDrive/Halomonas_Model_devt/HalomonasmodelV1.xml"

In [75]:
from cobra import Model, Reaction, Metabolite

In [76]:
model = cobra.io.read_sbml_model(Model_path)

In [77]:
model

Name,Halomonas_RAST
Memory address,0x07fbf7fdf3d90
Number of metabolites,1524
Number of reactions,2294
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


In [78]:
#model.summary()
model.get_metabolite_compartments()

{'C_c', 'C_e', 'C_p'}

#Inlude the pathway for PHB synthesis, not included in the draft model. 
There wwere no formulas in the BIGG databse for phb and hydroxybutyrl-CoA so i had to obtain it from the PubCHEM. 

Inorder to confirm these pathways i will run a visualisation in Escher in jupyter lab to see how connected the whole thing is. 

##Add  reaction PhbC

In [79]:


##Necessary Metabolites
#Create Phb metabolite
phb_c = Metabolite(
    'phb_c',
    formula='C4H8O3',
    name='Poly-beta-hydroxybutyrate',
    compartment='C_c')

# create metabolite 3hbcoa_R_c (written as h3bcoa for convenience )
h3bcoa__R_c =  Metabolite(
    '3hbcoa__R_c',
    formula='C25H42N7O18P3S',
    name='(R)-3-Hydroxybutyryl-CoA',
    compartment='C_c')

#get coa_c
coa_c = model.metabolites.get_by_id('coa_c')


#create a phb synthesis(Polymerization) Reaction 
PhbC_reaction = Reaction('PHBS_syn_1')
PhbC_reaction.name = 'PHB synthetase '
PhbC_reaction.subsystem = 'Butanote metabolism'
PhbC_reaction.lower_bound = -1000.0  # This is the default
PhbC_reaction.upper_bound = 1000.0  # This is the default


#ADD metabolites to PhC reactions 
PhbC_reaction.add_metabolites({
    h3bcoa__R_c: -1.0,
    coa_c: 1.0,
    phb_c: 1.0
})


#Gene reaction rule from RAST
PhbC_reaction.gene_reaction_rule = 'fig|6666666.858611.peg.1520'
PhbC_reaction.genes

frozenset({<Gene 6666666.858611.peg.1520 at 0x7fbf7f807e50>,
           <Gene fig at 0x7fbf7f807c10>})

##Add reaction Phb Sink

In [80]:
##DONT INCLUDE THIS

#Final metabolite (Phb) introduced as a Sink reaction because it is an intracellular polymer that accumulates in cell
#cytosolic PHB
#Add Phb sink reactions 
#phb_reaction = Reaction('sink_phb_c')
#phb_reaction.name = 'PHB sink transport reaction'
#phb_reaction.subsystem = 'Butanote metabolism'
#phb_reaction.lower_bound = 0.0  # This is the default check other snk reactions to know their bounds 
#phb_reaction.upper_bound = 1000.0  # This is the default

#ADD metabolites to Sink reactions
#phb_reaction.add_metabolites({
#    phb_c: -1.0
#})

#Gene reaction rule from Sink 
#phb_reaction.gene_reaction_rule = 'Spontaneous'
#phb_reaction.genes


##Add reaction PhbB

In [81]:
# add reaction for Acetoacetyl CoA reductase PhbB
#initialize metabolites
h_c = model.metabolites.get_by_id('h_c')
nadph_c = model.metabolites.get_by_id('nadph_c')
nadp_c = model.metabolites.get_by_id('nadp_c')
aacoa_c = model.metabolites.get_by_id('aacoa_c')

#Add PhbB reactions 
PhbB_reaction = Reaction('AACOAR_syn')
PhbB_reaction.name = 'Acetoacetyl CoA reductase'
PhbB_reaction.subsystem = 'Butanote metabolism'
PhbB_reaction.lower_bound = -1000.0  # This is the default check other snk reactions to know their bounds 
PhbB_reaction.upper_bound = 1000.0  # This is the default
#ADD metabolites to PhB reactions PhbC
PhbB_reaction.add_metabolites({
    h3bcoa__R_c: 1.0,
    aacoa_c: -1.0,
    h_c: -1.0,
    nadp_c: 1.0,
    nadph_c: -1.0
})
#Add Gene reaction rules
PhbB_reaction.gene_reaction_rule = '(fig|6666666.858611.peg.8 or fig|6666666.858611.peg.841)'
PhbB_reaction.genes


frozenset({<Gene 6666666.858611.peg.8 at 0x7fbf7fdb6410>,
           <Gene 6666666.858611.peg.841 at 0x7fbf7fdb61d0>,
           <Gene fig at 0x7fbf7fdb6d90>})

In [82]:
model

Name,Halomonas_RAST
Memory address,0x07fbf7fdf3d90
Number of metabolites,1524
Number of reactions,2294
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


##Add reaction PhaA

In [83]:
#PhaA reaction
PhbA_reaction = Reaction('KAT1')
PhbA_reaction.name = '3-ketoacyl-CoA thiolase'
PhbA_reaction.subsystem = 'Butanote metabolism'
PhbA_reaction.lower_bound = -1000.0  # This is the default check other snk reactions to know their bounds 
PhbA_reaction.upper_bound = 1000.0  # This is the default

#ADD metabolites to PhA reactions
PhbA_reaction.add_metabolites({
    aacoa_c: -1.0,
    aacoa_c: 2.0,
    coa_c: 1.0,
    })

#Add Gene reaction rules
PhbA_reaction.gene_reaction_rule = '(fig|6666666.858611.peg.1593 or fig|6666666.858611.peg.1761)'
PhbA_reaction.genes

frozenset({<Gene 6666666.858611.peg.1593 at 0x7fbf7f814810>,
           <Gene 6666666.858611.peg.1761 at 0x7fbf7f814b10>,
           <Gene fig at 0x7fbf7f814f90>})

##Add reaction Enoyl-CoA hydratase

In [84]:
#Enzyme name in BIGG database as 3-hydroxyacyl-CoA dehydratase
#so i be using this reaction considering the components are same 




##Adding all reactions to the model

In [85]:
#putting all the reactions in the model
model.add_reaction(PhbC_reaction)
model.add_reaction(PhbB_reaction)
model.add_reaction(PhbA_reaction)#already existed in the model reason for commenting it out
#model.add_reaction(Sink_reaction)
#The above line or this one
#strictly for testing. Want to see if the reaction stays a strict sink reaction or used as above, the difference that can ensue
#model.add_boundary(phb_c, type = 'sink', lb = 0.0, ub = 1000)

Ignoring reaction 'KAT1' since it already exists.


In [86]:
model.sinks

[]

In [87]:
model.reactions.EX_glc__D_e.lower_bound = -10.0
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,0.000000e+00
14GLUCANtexi,0.000000,0.000000e+00
1PPDCRc,0.000000,-4.207736e-16
2AGPE120tipp,0.000000,0.000000e+00
2AGPE140tipp,0.000000,0.000000e+00
...,...,...
sink_sheme_c,0.000000,0.000000e+00
Growth,1.001633,4.365539e-16
ATPM,0.000000,1.407645e-17
PHBS_syn_1,0.000000,0.000000e+00


In [88]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,31.36%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,7.924,6,68.64%
k_e,EX_k_e,0.1893,0,0.00%
mg2_e,EX_mg2_e,0.008415,0,0.00%


In [89]:
model.objective = 'PHBS_syn_1' #maximum possible PHB production under such glucose assimilation conditions
model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [90]:
model.objective = 'Growth'
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,31.36%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,7.924,6,68.64%
k_e,EX_k_e,0.1893,0,0.00%
mg2_e,EX_mg2_e,0.008415,0,0.00%


In [91]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model) #report these results for FVA using Glucose

,minimum,maximum
14GLUCANabcpp,0.000000,0.000000
14GLUCANtexi,0.000000,0.000000
1PPDCRc,0.000000,251.583229
2AGPE120tipp,0.000000,238.504201
2AGPE140tipp,0.000000,238.504201
...,...,...
sink_sheme_c,0.000000,33.201675
Growth,1.001633,1.001633
ATPM,0.000000,1000.000000
PHBS_syn_1,0.000000,0.000000


In [92]:
#cobra.io.write_sbml_model(model, "/content/drive/MyDrive/Halomonas_Model_devt/(+PHBrxns)_Model.xml")
#cobra.io.save_json_model(model, "/content/drive/MyDrive/Model_devt_Ch/FinalModel_V2.json")

In [93]:
%%capture
model.boundary

#Test the Pathway

In [94]:
model.metabolites.get_by_id('3hbcoa__R_c')

Metabolite identifier,3hbcoa__R_c
Name,(R)-3-Hydroxybutyryl-CoA
Memory address,0x07fbf7f807cd0
Formula,C25H42N7O18P3S
Compartment,C_c
In 2 reaction(s),"PHBS_syn_1, AACOAR_syn"


In [95]:
#change the carbon source to see how it affects the roduction of PHB by seeing the changes in flux variablity.
medium = model.medium
medium['EX_fru_e'] = 10.0
model.medium = medium
model.reactions.EX_glc__D_e.lower_bound = 0.0
model.reactions.EX_fru_e.lower_badound = -10.0
#model.reactions.EX_nh4_e.lower_bound = 1000.0

In [96]:
model.medium

{'EX_asn__L_e': 1000.0,
 'EX_bz_e': 9.7e-05,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_fru_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [97]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,31.36%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
fru_e,EX_fru_e,7.924,6,68.64%
k_e,EX_k_e,0.1893,0,0.00%
mg2_e,EX_mg2_e,0.008415,0,0.00%


In [98]:
flux_variability_analysis(model)

,minimum,maximum
14GLUCANabcpp,0.000000,0.000000
14GLUCANtexi,0.000000,0.000000
1PPDCRc,0.000000,251.583229
2AGPE120tipp,0.000000,238.504201
2AGPE140tipp,0.000000,238.504201
...,...,...
sink_sheme_c,0.000000,33.201675
Growth,1.001633,1.001633
ATPM,0.000000,1000.000000
PHBS_syn_1,0.000000,0.000000


In [99]:
#was no phb produced in glucose only, nitrogen limiting conditions, fructose and  glucose, fructose alone. No phb was produced, we make sure tomorrow that the connectivity of the model has 
#not been broken with the reaction, maybe it doesnt connect with other components of the model. use escher to do this
#Try to add the other Fatty acid reactions. FInd if they are in the model already(my guess is there is)
#see how they all connect

In [100]:
cobra.flux_analysis.find_blocked_reactions(model)

['14GLUCANabcpp',
 '14GLUCANtexi',
 '2DHGLCK',
 '2HH24DDH',
 '2HH24DDH1',
 '2HXMPt6',
 '2PGLYCt6',
 '34DHCINMtex',
 '34DHCINMtpp',
 '34dhpactex',
 '3HOXPACt2pp',
 '3hoxpactex',
 '3HPAOX',
 '3MBt2pp',
 '3MBt4pp',
 '3MBtex',
 '42A12BOOXpp',
 '4CMCOAS',
 '4H2KPILY',
 '4HOXPACMOF',
 '4HOXPACt2pp',
 'AAMYL',
 'ABUTtex',
 'ACGApts',
 'ACGAptspp',
 'ACGAtex',
 'ACMANApts',
 'ACMANAptspp',
 'ACMANAtex',
 'ACMUMptspp',
 'ACMUMtex',
 'ACTD',
 'ACTD2',
 'ACTDa',
 'ADNt2',
 'ALAGLUE',
 'ALCD3',
 'ALCD4',
 'ALDD1',
 'ALDD19xr',
 'ALDD31',
 'ALDD31_1',
 'ALDD4',
 'ALDD6',
 'AMMQT8_2',
 'AMPTASEPG',
 'ARABDI',
 'ARAI',
 'ASO3t8pp',
 'ASO3tex',
 'ASO4rt2rpp',
 'ASO4tex',
 'ASR',
 'BETALDHx',
 'BETALDHy',
 'BHMT',
 'BMOCOS',
 'BMOGDS1',
 'BMOGDS2',
 'BTDD_RR',
 'BTOHt2rpp',
 'BTOHtex',
 'BUTCT',
 'CACOAHA',
 'CAFFCOA',
 'CHOLD',
 'CHOLD2',
 'CHOLSabc',
 'CHOLSabc_1',
 'CHOLS_ex',
 'CHOLSH',
 'CHTBSptspp',
 'CHTBStex',
 'CMCMSAD',
 'CMHMI',
 'COCOAHA',
 'CPMPS',
 'CYNTAH',
 'CYNTt2',
 'CYNTt2pp',
 'CYNT

In [101]:
model.reactions.PHBS_syn_1

Reaction identifier,PHBS_syn_1
Name,PHB synthetase
Memory address,0x07fbf7f807790
Stoichiometry,3hbcoa__R_c <=> coa_c + phb_c (R)-3-Hydroxybutyryl-CoA <=> Coenzyme A + Poly-beta-hydroxybutyrate
GPR,fig|6666666.858611.peg.1520
Lower bound,-1000.0
Upper bound,1000.0


In [102]:
model.reactions.AACOAR_syn

Reaction identifier,AACOAR_syn
Name,Acetoacetyl CoA reductase
Memory address,0x07fbf7fe2bbd0
Stoichiometry,aacoa_c + h_c + nadph_c <=> 3hbcoa__R_c + nadp_c Acetoacetyl-CoA + H+ + Nicotinamide adenine dinucleotide phosphate - reduced <=> (R)-3-Hydroxybutyryl-CoA + Nicotinamide adenine dinucleotide phosphate
GPR,(fig|6666666.858611.peg.8 or fig|6666666.858611.peg.841)
Lower bound,-1000.0
Upper bound,1000.0


In [103]:
model.reactions.KAT1.lower_bound = -1000.0
model.reactions.KAT1

Reaction identifier,KAT1
Name,3-ketoacyl-CoA thiolase
Memory address,0x07fbf7ef72e50
Stoichiometry,aacoa_c + coa_c <=> 2.0 accoa_c Acetoacetyl-CoA + Coenzyme A <=> 2.0 Acetyl-CoA
GPR,fig_6666666_879901_peg_1830 or fig_6666666_879901_peg_2360
Lower bound,-1000.0
Upper bound,1000.0


In [104]:
model.metabolites.aacoa_c

Metabolite identifier,aacoa_c
Name,Acetoacetyl-CoA
Memory address,0x07fbf801fe690
Formula,C25H36N7O18P3S
Compartment,C_c
In 9 reaction(s),"HACD1i, KAT1, FAO4, ACACT1r, OCOAT1, ACACCT, HMGCOASi, HACD1, AACOAR_syn"


In [105]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,31.36%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
fru_e,EX_fru_e,7.924,6,68.64%
k_e,EX_k_e,0.1893,0,0.00%
mg2_e,EX_mg2_e,0.008415,0,0.00%


In [106]:
consistent_model = cobra.flux_analysis.fastcc(model)

In [107]:
model.objective = "Growth"
consistent_model.optimize()

,fluxes,reduced_costs
1PPDCRc,0.000000,-6.384171e-17
2AGPE120tipp,0.000000,-2.341992e-12
2AGPE140tipp,0.000000,0.000000e+00
2AGPE141tipp,0.000000,-2.019484e-28
2AGPE160tipp,0.000000,2.051883e-12
...,...,...
sink_lipopb_c,0.000000,0.000000e+00
sink_mththf_c,0.000000,0.000000e+00
sink_sheme_c,0.000000,0.000000e+00
Growth,1.001633,3.551668e-16


In [108]:
model.reactions.EX_bz_e.lower_bound = -0.0
medium = model.medium
medium["EX_bz_e"] = 9.7e-05
model.medium = medium
model.medium

{'EX_asn__L_e': 1000.0,
 'EX_bz_e': 9.7e-05,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_fru_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [109]:
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,0.0
14GLUCANtexi,0.000000,0.0
1PPDCRc,0.000000,0.0
2AGPE120tipp,0.000000,0.0
2AGPE140tipp,0.000000,0.0
...,...,...
sink_sheme_c,0.000000,0.0
Growth,1.001633,0.0
ATPM,0.000000,0.0
PHBS_syn_1,0.000000,-0.0


In [110]:
model.add_boundary(model.metabolites.get_by_id("phb_c"), type="demand")

Reaction identifier,DM_phb_c
Name,Poly-beta-hydroxybutyrate demand
Memory address,0x07fbf7ee75f50
Stoichiometry,phb_c --> Poly-beta-hydroxybutyrate -->
GPR,
Lower bound,0
Upper bound,1000.0


In [111]:
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,0.0
14GLUCANtexi,0.000000,0.0
1PPDCRc,0.000000,0.0
2AGPE120tipp,0.000000,0.0
2AGPE140tipp,0.000000,0.0
...,...,...
Growth,1.001633,0.0
ATPM,0.000000,0.0
PHBS_syn_1,0.000000,-0.0
AACOAR_syn,0.000000,0.0


In [112]:
model.objective = "PHBS_syn_1"
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.00000,6.661338e-16
14GLUCANtexi,0.00000,0.000000e+00
1PPDCRc,0.00000,-1.175610e+00
2AGPE120tipp,0.00000,0.000000e+00
2AGPE140tipp,0.00000,-3.853659e-01
...,...,...
Growth,0.00000,-2.285292e+01
ATPM,0.00000,-1.926829e-01
PHBS_syn_1,324.51222,0.000000e+00
AACOAR_syn,324.51222,0.000000e+00


In [113]:
model.objective = "Growth"
model.optimize()

,fluxes,reduced_costs
14GLUCANabcpp,0.000000,0.0
14GLUCANtexi,0.000000,0.0
1PPDCRc,0.000000,0.0
2AGPE120tipp,0.000000,0.0
2AGPE140tipp,0.000000,0.0
...,...,...
Growth,1.001633,0.0
ATPM,0.000000,0.0
PHBS_syn_1,309.984880,0.0
AACOAR_syn,309.984880,0.0


In [114]:
#use this notebook to fix things around here. solves the polymer production problem. demand plus compartment corection.
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
asn__L_e,EX_asn__L_e,5.431,4,31.36%
bz_e,EX_bz_e,9.7E-05,7,0.00%
ca2_e,EX_ca2_e,0.005049,0,0.00%
cl_e,EX_cl_e,0.005049,0,0.00%
cobalt2_e,EX_cobalt2_e,9.7E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006877,0,0.00%
fe3_e,EX_fe3_e,0.01409,0,0.00%
fru_e,EX_fru_e,7.924,6,68.64%
k_e,EX_k_e,0.1893,0,0.00%
mg2_e,EX_mg2_e,0.008415,0,0.00%


In [115]:
cobra.io.write_sbml_model(model, "/content/drive/MyDrive/Halomonas_Model_devt/(+DemandPHBrxns)_Model.xml")
#cobra.io.save_json_model(model, "/content/drive/MyDrive/Model_devt_Ch/FinalModel_V2.json")

In [116]:
model

Name,Halomonas_RAST
Memory address,0x07fbf7fdf3d90
Number of metabolites,1526
Number of reactions,2297
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


In [118]:
model.metabolites.phb_c

Metabolite identifier,phb_c
Name,Poly-beta-hydroxybutyrate
Memory address,0x07fbf7f807d10
Formula,C4H8O3
Compartment,C_c
In 2 reaction(s),"PHBS_syn_1, DM_phb_c"


In [122]:
model.get_metabolite_compartments()

{'C_c', 'C_e', 'C_p'}